# Cross-currency swaps

At this time, there’s no instrument class in the library modeling cross-currency swaps. However, it’s possible to calculate their value by working with cashflows. Here is a short example.

In [1]:
import QuantLib as ql
import pandas as pd

today = ql.Date(27, ql.October, 2021)
ql.Settings.instance().evaluationDate = today

bps = 1e-4

# Sample data

In [2]:
sample_rates = pd.DataFrame(
    [
        (ql.Date(27, 10, 2021), 0.0229, 0.0893, -0.5490, -0.4869),
        (ql.Date(27, 1, 2022), 0.0645, 0.1059, -0.5584, -0.5057),
        (ql.Date(27, 4, 2022), 0.0414, 0.1602, -0.5480, -0.5236),
        (ql.Date(27, 10, 2022), 0.1630, 0.2601, -0.5656, -0.5030),
        (ql.Date(27, 10, 2023), 0.4639, 0.6281, -0.4365, -0.3468),
        (ql.Date(27, 10, 2024), 0.7187, 0.9270, -0.3500, -0.2490),
        (ql.Date(27, 10, 2025), 0.9056, 1.1257, -0.3041, -0.1590),
        (ql.Date(27, 10, 2026), 1.0673, 1.2821, -0.2340, -0.0732),
        (ql.Date(27, 10, 2027), 1.1615, 1.3978, -0.1690, -0.0331),
        (ql.Date(27, 10, 2028), 1.2326, 1.4643, -0.1041, 0.0346),
        (ql.Date(27, 10, 2029), 1.3050, 1.5589, -0.0070, 0.1263),
        (ql.Date(27, 10, 2030), 1.3584, 1.5986, 0.0272, 0.1832),
        (ql.Date(27, 10, 2031), 1.4023, 1.6488, 0.0744, 0.2599),
        (ql.Date(27, 10, 2036), 1.5657, 1.8136, 0.3011, 0.4406),
        (ql.Date(27, 10, 2041), 1.6191, 1.8749, 0.3882, 0.5331),
        (ql.Date(27, 10, 2046), 1.6199, 1.8701, 0.3762, 0.5225),
        (ql.Date(27, 10, 2051), 1.6208, 1.8496, 0.3401, 0.4926),
    ],
    columns=["date", "SOFR", "USDLibor3M", "ESTR", "Euribor3M"],
)

In [3]:
def sample_curve(tag):
    curve = ql.ZeroCurve(
        sample_rates["date"], sample_rates[tag] / 100, ql.Actual365Fixed()
    )
    return ql.YieldTermStructureHandle(curve)

# Const-notional cross-currency swaps

In [4]:
euribor3M_curve = sample_curve("Euribor3M")
euribor3M = ql.Euribor(ql.Period(3, ql.Months), euribor3M_curve)
usdlibor3M_curve = sample_curve("USDLibor3M")
usdlibor3M = ql.USDLibor(ql.Period(3, ql.Months), usdlibor3M_curve)

In [6]:
notional = 1_000_000
fx_0 = 0.85
spread = 50.0 * bps

In [7]:
calendar = ql.UnitedStates(ql.UnitedStates.FederalReserve)
start_date = calendar.advance(today, ql.Period(2, ql.Days))
end_date = calendar.advance(start_date, ql.Period(5, ql.Years))
tenor = ql.Period(3, ql.Months)
rule = ql.DateGeneration.Forward
convention = ql.Following
end_of_month = False

schedule = ql.Schedule(
    start_date,
    end_date,
    tenor,
    calendar,
    convention,
    convention,
    rule,
    end_of_month,
)

In [8]:
usd_leg = (
    (ql.SimpleCashFlow(-notional, schedule[0]),)
    + ql.IborLeg(
        nominals=[notional],
        schedule=schedule,
        index=usdlibor3M,
        spreads=[spread],
    )
    + (ql.SimpleCashFlow(notional, schedule[-1]),)
)

In [10]:
eur_leg = (
    (ql.SimpleCashFlow(-notional * fx_0, schedule[0]),)
    + ql.IborLeg(nominals=[notional * fx_0], schedule=schedule, index=euribor3M)
    + (ql.SimpleCashFlow(notional * fx_0, schedule[-1]),)
)

Now, we can get the NPV of each leg in its own currency by discounting them with the corresponding overnight curve:

In [11]:
sofr_curve = sample_curve("SOFR")
usd_npv = ql.CashFlows.npv(usd_leg, sofr_curve, True)
usd_npv

35427.65532574046

In [12]:
estr_curve = sample_curve("ESTR")
eur_npv = ql.CashFlows.npv(eur_leg, estr_curve, True)
eur_npv

6908.723028828739

In [13]:
ql.CashFlows.npv(eur_leg, estr_curve, True) / fx_0

8127.90944568087

In [14]:
def cashflow_data(leg):
    data = []
    for cf in sorted(leg, key=lambda c: c.date()):
        coupon = ql.as_floating_rate_coupon(cf)
        if coupon is None:
            data.append((cf.date(), None, None, cf.amount()))
        else:
            data.append(
                (
                    coupon.date(),
                    coupon.nominal(),
                    coupon.rate(),
                    coupon.amount(),
                )
            )
    return pd.DataFrame(
        data, columns=["date", "nominal", "rate", "amount"]
    ).style.format({"amount": "{:.2f}", "nominal": "{:.2f}", "rate": "{:.2%}"})



Here are the cashflows in USD…



In [15]:
cashflow_data(usd_leg)

,date,nominal,rate,amount
0,"October 29th, 2021",nan,nan%,-1000000.00
1,"January 31st, 2022",1000000.00,0.61%,1585.56
2,"April 29th, 2022",1000000.00,0.72%,1750.57
3,"July 29th, 2022",1000000.00,0.81%,2040.59
4,"October 31st, 2022",1000000.00,0.91%,2386.92
5,"January 30th, 2023",1000000.00,1.22%,3080.34
6,"May 1st, 2023",1000000.00,1.40%,3541.30
7,"July 31st, 2023",1000000.00,1.58%,3999.86
8,"October 30th, 2023",1000000.00,1.76%,4444.64
9,"January 29th, 2024",1000000.00,1.79%,4518.96


In [16]:
cashflow_data(eur_leg)

,date,nominal,rate,amount
0,"October 29th, 2021",nan,nan%,-850000.00
1,"January 31st, 2022",850000.00,-0.50%,-1108.91
2,"April 29th, 2022",850000.00,-0.53%,-1109.57
3,"July 29th, 2022",850000.00,-0.49%,-1042.88
4,"October 31st, 2022",850000.00,-0.46%,-1020.88
5,"January 30th, 2023",850000.00,-0.30%,-644.90
6,"May 1st, 2023",850000.00,-0.22%,-479.05
7,"July 31st, 2023",850000.00,-0.15%,-314.08
8,"October 30th, 2023",850000.00,-0.07%,-158.20
9,"January 29th, 2024",850000.00,-0.12%,-266.58


In [17]:
NPV = usd_npv - eur_npv / fx_0
NPV

27299.74588005959

# Mark-to-market cross-currency swaps

In this more common kind of cross-currency swaps, the notionals are rebalanced at each coupon date so that their value remain the same (according, of course, to the value of the FX rate at each coupon start).

In order to model this rebalancing feature we will need to estimate the FX rates in the future, the corresponding notionals in Euro for the floating cashflows, and the amounts exchanged due to rebalancing.

The future FX rates can be forecast from the discount curves, since they model the cost of money. We’ll write a convenience function to extract it at any given date:

In [18]:
def FX(date):
    return fx_0 * sofr_curve.discount(date) / estr_curve.discount(date)

In [24]:
start_dates = list(schedule)[:-1]

notionals = [notional * FX(d) for d in start_dates]


rebalancing_cashflows = []
for i in range(len(notionals) - 1):
    rebalancing_cashflows.append(
        ql.SimpleCashFlow(notionals[i] - notionals[i + 1], schedule[i + 1])
    )

In [27]:
print(start_dates)
print(notionals)
print(rebalancing_cashflows)

[Date(29,10,2021), Date(31,1,2022), Date(29,4,2022), Date(29,7,2022), Date(31,10,2022), Date(30,1,2023), Date(1,5,2023), Date(31,7,2023), Date(30,10,2023), Date(29,1,2024), Date(29,4,2024), Date(29,7,2024), Date(29,10,2024), Date(29,1,2025), Date(29,4,2025), Date(29,7,2025), Date(29,10,2025), Date(29,1,2026), Date(29,4,2026), Date(29,7,2026)]
[849973.3123427323, 848611.8977686284, 847471.7102737654, 845782.8807591915, 843745.972788649, 841756.2314215006, 839591.8474141428, 837254.1930492707, 834745.2933863958, 832084.2529027408, 829258.650724047, 826270.2226822922, 823088.945483854, 819922.5791544188, 816695.2396430873, 813303.1670498879, 809752.6285092995, 806492.6715342533, 803225.8167783241, 799845.5319662248]
[<QuantLib.QuantLib.SimpleCashFlow; proxy of <Swig Object of type 'ext::shared_ptr< SimpleCashFlow > *' at 0x0000020FBC9EA7F0> >, <QuantLib.QuantLib.SimpleCashFlow; proxy of <Swig Object of type 'ext::shared_ptr< SimpleCashFlow > *' at 0x0000020FBC9EA400> >, <QuantLib.QuantLib

Finally, we can create the two legs and price them. The USD leg is as before, since its notional doesn’t change:

In [28]:
usd_leg = (
    (ql.SimpleCashFlow(-notional, schedule[0]),)
    + ql.IborLeg(
        nominals=[notional],
        schedule=schedule,
        index=usdlibor3M,
        spreads=[spread],
    )
    + (ql.SimpleCashFlow(notional, schedule[-1]),)
)

In [30]:
eur_leg = (
    [ql.SimpleCashFlow(-notionals[0], schedule[0])]
    + list(ql.IborLeg(nominals=notionals, schedule=schedule, index=euribor3M))
    + rebalancing_cashflows
    + [ql.SimpleCashFlow(notionals[-1], schedule[-1])]
)

In [31]:
cashflow_data(eur_leg)

,date,nominal,rate,amount
0,"October 29th, 2021",nan,nan%,-849973.31
1,"January 31st, 2022",849973.31,-0.50%,-1108.87
2,"January 31st, 2022",nan,nan%,1361.41
3,"April 29th, 2022",848611.90,-0.53%,-1107.76
4,"April 29th, 2022",nan,nan%,1140.19
5,"July 29th, 2022",847471.71,-0.49%,-1039.78
6,"July 29th, 2022",nan,nan%,1688.83
7,"October 31st, 2022",845782.88,-0.46%,-1015.82
8,"October 31st, 2022",nan,nan%,2036.91
9,"January 30th, 2023",843745.97,-0.30%,-640.16


In [32]:
usd_npv = ql.CashFlows.npv(usd_leg, sofr_curve, True)
usd_npv

35427.65532574046

In [33]:
eur_npv = ql.CashFlows.npv(eur_leg, estr_curve, True)
eur_npv

6678.7560644123005

In [34]:
NPV = usd_npv - eur_npv / fx_0
NPV

27570.29524996128